<div class = "alert alert-danger">
Nice report and very good code, nice coding skills. The problem is interesting, but there is something that you could improve in the report: better structuring. You mention in the introduction that you are going to explore the data: "it will give us insights if sellers are pricing their cars at a reasonable rate depending on the aspects that where discussed before like mileage, age and brand of the car". That is fine, but a bit too abstract. Before going into more detail in the report, I would like to see concrete questions that you want to solve with the data, and how they can be translated into actionable knowledge and/or recommendations. After you have stated those questions, then you start explaining results. The questions should actually be the title of subsections in your report. Right now, you describe what you are doing (Time online vs km, age vs fuel type) but not a concrete thing or question you want to answer in advance. The conclusions section is a bit disconnected from your data, as you only say most sold used cars are mainly from Germany. 

<b>GRADE: </b> 9.0
</div>

# AUTOS

1. PySpark **environment setup**
2. Data source and **Spark data abstraction** (DataFrame) **set up**
3. Data set **metadata analysis**:
  1. Display **schema and size** of the DataFrame
  2. Get one or multiple **random samples** from the data set to better understand what the data is all about
4. **Price** and **Year of Registration** Analysis
  1. Describe price and Year of Registration
  2. Filtering Dataset by Price and Year of Registration conditions
5. **Dropping** misleading variables
6. **Creating** new variables
  1. Date of Registration
  2. Age of Car (in years)
  3. Time of ad online
7. Working with **NAs** and **Age of Car**
  1. Checking for NAs
  2. Dropping NAs
  3. Checking
  4. Removing ads where Age of Car < 0
  3. Number of Ads remaining 
8. Overview of  **Price vs Count**
  1. Checking the Cheapest, Expensive, Most sold and Least sold brands
  2. Filtering illogical high prices of cars
9. Overview of  **Timeonline vs Kilometers**
  1. Checking for Least and Most car online and distance covered
10. Overview of **Age vs Fuel Type** 
  1. Checking for Newest and Oldest Car with the most used fuel types
  2. Age of Car by Brand
  3. Fuel Type Average Price
  4. Count of different types of gearbox
  
 ## Link to Dataset: https://www.kaggle.com/orgesleka/used-cars-database

     
  ## The variables 
    dateCrawled : when this ad was first crawled, all field-values are taken from this date
    name : "name" of the car
    seller : private or dealer
    offerType 
    price : the price on the ad to sell the car
    abtest
    vehicleType
    yearOfRegistration : at which year the car was first registered
    gearbox
    powerPS : power of the car in PS
    model
    kilometer : how many kilometers the car has driven
    monthOfRegistration : at which month the car was first registered
    fuelType : benzene, gasoline, hybrid,elektric..
    brand: audi, bmw, jaguar, nissan, porsche...
    notRepairedDamage : if the car has a damage which is not repaired yet
    dateCreated : the date for which the ad at ebay was created
    nrOfPictures : number of pictures in the ad 
    postalCode
    lastSeenOnline : when the crawler saw this ad last online


The dataset consists of over 370,000 used cars scraped with Scrapy from Ebay-Kleinanzeigen. An analysis will be done to create a cleaned dataset that will help us understand more how logical the clients are pricing their cars on Ebay.


## 1- PySpark **environment setup**

In [2]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## 2. Data source and Spark data abstraction (DataFrame) setup

In [6]:
autosDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("datasets/autos.csv")

## 3. Data set metadata analysis
#### A. Display schema and size of the DataFrame

In [7]:
from IPython.display import display, Markdown

autosDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % autosDF.count()))

root
 |-- dateCrawled: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: integer (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: integer (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- dateCreated: timestamp (nullable = true)
 |-- nrOfPictures: integer (nullable = true)
 |-- postalCode: integer (nullable = true)
 |-- lastSeen: timestamp (nullable = true)



This DataFrame has **371824 rows**.

#### B. Get one or multiple random samples from the data set

In [8]:
autosDF.sample(False, 0.1).show(3,truncate= False, vertical =True)

-RECORD 0-------------------------------------------
 dateCrawled         | 2016-03-24 11:52:17          
 name                | Golf_3_1.6                   
 seller              | privat                       
 offerType           | Angebot                      
 price               | 480                          
 abtest              | test                         
 vehicleType         | null                         
 yearOfRegistration  | 1993                         
 gearbox             | manuell                      
 powerPS             | 0                            
 model               | golf                         
 kilometer           | 150000                       
 monthOfRegistration | 0                            
 fuelType            | benzin                       
 brand               | volkswagen                   
 notRepairedDamage   | null                         
 dateCreated         | 2016-03-24 00:00:00          
 nrOfPictures        | 0                      

## 4.  Price and Year of Registration Analysis
#### A. Describe Price and Year of Registration Analysis

In [9]:
print("Price Summary :")
autosDF.describe('Price').show()
print("Year of Registration Summary :")
autosDF.describe('yearOfRegistration').show()

Price Summary :
+-------+------------------+
|summary|             Price|
+-------+------------------+
|  count|            371823|
|   mean|17286.338865535483|
| stddev|3586530.1840677853|
|    min|                 0|
|    max|        2147483647|
+-------+------------------+

Year of Registration Summary :
+-------+------------------+
|summary|yearOfRegistration|
+-------+------------------+
|  count|            371823|
|   mean|2004.5767206439623|
| stddev|  92.8299905430405|
|    min|              1000|
|    max|              9999|
+-------+------------------+



#### B. Filtering Dataset by Price and Year of Registration conditions

In [10]:
autos_DF = autosDF.where("Price > 500 and Price < 100000")
autos_DF = autos_DF.where("yearOfRegistration > 1900 and yearOfRegistration < 2020")
autos_DF.cache()

DataFrame[dateCrawled: timestamp, name: string, seller: string, offerType: string, price: int, abtest: string, vehicleType: string, yearOfRegistration: int, gearbox: string, powerPS: int, model: string, kilometer: int, monthOfRegistration: int, fuelType: string, brand: string, notRepairedDamage: string, dateCreated: timestamp, nrOfPictures: int, postalCode: int, lastSeen: timestamp]

In [11]:
print("Price Summary :")
autos_DF.describe('Price').show()
print("Year of Registration Summary :")
autos_DF.describe('yearOfRegistration').show()

Price Summary :
+-------+-----------------+
|summary|            Price|
+-------+-----------------+
|  count|           329568|
|   mean|6285.719942470143|
| stddev|7688.797185591162|
|    min|              501|
|    max|            99999|
+-------+-----------------+

Year of Registration Summary :
+-------+------------------+
|summary|yearOfRegistration|
+-------+------------------+
|  count|            329568|
|   mean|2003.8476824206234|
| stddev| 7.440211442291847|
|    min|              1910|
|    max|              2019|
+-------+------------------+



In [12]:
autos_DF.show(n=20, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------------------
 dateCrawled         | 2016-03-24 10:58:45                                               
 name                | A5_Sportback_2.7_Tdi                                              
 seller              | privat                                                            
 offerType           | Angebot                                                           
 price               | 18300                                                             
 abtest              | test                                                              
 vehicleType         | coupe                                                             
 yearOfRegistration  | 2011                                                              
 gearbox             | manuell                                                           
 powerPS             | 190                                                               
 model    

## 5. Dropping misleading variables

In [13]:
autos_DF =autos_DF.drop("dateCrawled").drop("name").drop("abtest").drop("offerType")\
                  .drop("Seller").drop("powerPS").drop("nrofPictures").drop("postalCode").drop("model")
autos_DF.show(n=20, truncate=False, vertical=True)

-RECORD 0----------------------------------
 price               | 18300               
 vehicleType         | coupe               
 yearOfRegistration  | 2011                
 gearbox             | manuell             
 kilometer           | 125000              
 monthOfRegistration | 5                   
 fuelType            | diesel              
 brand               | audi                
 notRepairedDamage   | ja                  
 dateCreated         | 2016-03-24 00:00:00 
 lastSeen            | 2016-04-07 01:46:50 
-RECORD 1----------------------------------
 price               | 9800                
 vehicleType         | suv                 
 yearOfRegistration  | 2004                
 gearbox             | automatik           
 kilometer           | 125000              
 monthOfRegistration | 8                   
 fuelType            | diesel              
 brand               | jeep                
 notRepairedDamage   | null                
 dateCreated         | 2016-03-1

### Importing the needed functions and types

In [14]:
from pyspark.sql.functions import lit, concat, datediff, bround, max, asc, desc, count, when, min, max, stddev, to_timestamp, avg,countDistinct, col
from pyspark.sql.types import DateType

from pyspark.sql import functions as sf

## 6. Creating new variables
##### A. Date of Registration
    

In [15]:
autos_DF = autos_DF.withColumn('DateofRegistration', 
                    sf.concat(sf.col('yearofRegistration'), sf.lit('-'), sf.col('monthofRegistration'),\
                                sf.lit('-'), lit("1") ))

autos_DF.show(n=20, truncate=False, vertical=True)

-RECORD 0----------------------------------
 price               | 18300               
 vehicleType         | coupe               
 yearOfRegistration  | 2011                
 gearbox             | manuell             
 kilometer           | 125000              
 monthOfRegistration | 5                   
 fuelType            | diesel              
 brand               | audi                
 notRepairedDamage   | ja                  
 dateCreated         | 2016-03-24 00:00:00 
 lastSeen            | 2016-04-07 01:46:50 
 DateofRegistration  | 2011-5-1            
-RECORD 1----------------------------------
 price               | 9800                
 vehicleType         | suv                 
 yearOfRegistration  | 2004                
 gearbox             | automatik           
 kilometer           | 125000              
 monthOfRegistration | 8                   
 fuelType            | diesel              
 brand               | jeep                
 notRepairedDamage   | null     

##### B. Age of Car in years

In [16]:
autos_DF =autos_DF.drop("yearofRegistration","monthofRegistration")
autos_DF = autos_DF.withColumn('DateofRegistration', autos_DF['DateofRegistration'].cast(DateType()))
autos_DF = autos_DF.withColumn('dateCreated', autos_DF['dateCreated'].cast(DateType()))
autos_DF.show(n=20, truncate=False, vertical=True)

-RECORD 0---------------------------------
 price              | 18300               
 vehicleType        | coupe               
 gearbox            | manuell             
 kilometer          | 125000              
 fuelType           | diesel              
 brand              | audi                
 notRepairedDamage  | ja                  
 dateCreated        | 2016-03-24          
 lastSeen           | 2016-04-07 01:46:50 
 DateofRegistration | 2011-05-01          
-RECORD 1---------------------------------
 price              | 9800                
 vehicleType        | suv                 
 gearbox            | automatik           
 kilometer          | 125000              
 fuelType           | diesel              
 brand              | jeep                
 notRepairedDamage  | null                
 dateCreated        | 2016-03-14          
 lastSeen           | 2016-04-05 12:47:46 
 DateofRegistration | 2004-08-01          
-RECORD 2---------------------------------
 price     

In [17]:
DateDiff = datediff(sf.col("dateCreated"),sf.col("DateofRegistration"))/365
autos_DF = autos_DF.withColumn('AgeofCar_inyears', DateDiff)
autos_DF = autos_DF.withColumn('AgeofCar_inyears', sf.round('AgeofCar_inyears', 2))
autos_DF.show(n=20, truncate=False, vertical=True)

-RECORD 0---------------------------------
 price              | 18300               
 vehicleType        | coupe               
 gearbox            | manuell             
 kilometer          | 125000              
 fuelType           | diesel              
 brand              | audi                
 notRepairedDamage  | ja                  
 dateCreated        | 2016-03-24          
 lastSeen           | 2016-04-07 01:46:50 
 DateofRegistration | 2011-05-01          
 AgeofCar_inyears   | 4.9                 
-RECORD 1---------------------------------
 price              | 9800                
 vehicleType        | suv                 
 gearbox            | automatik           
 kilometer          | 125000              
 fuelType           | diesel              
 brand              | jeep                
 notRepairedDamage  | null                
 dateCreated        | 2016-03-14          
 lastSeen           | 2016-04-05 12:47:46 
 DateofRegistration | 2004-08-01          
 AgeofCar_i

##### C. Time of ad online

In [18]:
DateDiff = datediff(sf.col("lastSeen"),sf.col("dateCreated"))
autos_DF = autos_DF.withColumn('Timeonline_indays', DateDiff)
autos_DF.show(n=20, truncate=False, vertical=True)

-RECORD 0---------------------------------
 price              | 18300               
 vehicleType        | coupe               
 gearbox            | manuell             
 kilometer          | 125000              
 fuelType           | diesel              
 brand              | audi                
 notRepairedDamage  | ja                  
 dateCreated        | 2016-03-24          
 lastSeen           | 2016-04-07 01:46:50 
 DateofRegistration | 2011-05-01          
 AgeofCar_inyears   | 4.9                 
 Timeonline_indays  | 14                  
-RECORD 1---------------------------------
 price              | 9800                
 vehicleType        | suv                 
 gearbox            | automatik           
 kilometer          | 125000              
 fuelType           | diesel              
 brand              | jeep                
 notRepairedDamage  | null                
 dateCreated        | 2016-03-14          
 lastSeen           | 2016-04-05 12:47:46 
 DateofRegi

## 7. Working with NAs
##### A. Checking for NAs

In [19]:
print("Checking for nulls on columns :")
autos_DF.select([count(when(sf.col(c).isNull(), c)).alias(c) for c in ["price","vehicleType","gearbox",\
            "kilometer", "fuelType","brand","notRepairedDamage", "DateofRegistration", "AgeofCar_inyears"]]).show()

Checking for nulls on columns :
+-----+-----------+-------+---------+--------+-----+-----------------+------------------+----------------+
|price|vehicleType|gearbox|kilometer|fuelType|brand|notRepairedDamage|DateofRegistration|AgeofCar_inyears|
+-----+-----------+-------+---------+--------+-----+-----------------+------------------+----------------+
|    0|      26501|  13138|        0|   22914|    0|            53828|             23930|           23930|
+-----+-----------+-------+---------+--------+-----+-----------------+------------------+----------------+



##### B. Removing NAs

In [20]:
autos_DF = autos_DF.na.drop(how='any')


In [21]:
print("Checking for nulls on columns :")
autos_DF.select([count(when(sf.col(c).isNull(), c)).alias(c) for c in ["price","vehicleType","gearbox",\
            "kilometer", "fuelType","brand","notRepairedDamage", "DateofRegistration", "AgeofCar_inyears"]]).show()

Checking for nulls on columns :
+-----+-----------+-------+---------+--------+-----+-----------------+------------------+----------------+
|price|vehicleType|gearbox|kilometer|fuelType|brand|notRepairedDamage|DateofRegistration|AgeofCar_inyears|
+-----+-----------+-------+---------+--------+-----+-----------------+------------------+----------------+
|    0|          0|      0|        0|       0|    0|                0|                 0|               0|
+-----+-----------+-------+---------+--------+-----+-----------------+------------------+----------------+



##### C. Checking for newest cars

In [22]:
autos_DF.groupBy('AgeofCar_inyears').agg(count('AgeofCar_inyears').alias('count'))\
                                                                        .orderBy(asc('AgeofCar_inyears')).show()

+----------------+-----+
|AgeofCar_inyears|count|
+----------------+-----+
|           -2.73|    1|
|            -2.7|    1|
|           -2.15|    1|
|           -1.61|    1|
|           -1.36|    1|
|            -1.3|    1|
|           -1.22|    1|
|           -1.19|    1|
|           -1.15|    1|
|           -0.67|    2|
|           -0.45|    1|
|           -0.23|    1|
|           -0.08|    1|
|           -0.07|    1|
|           -0.05|    1|
|           -0.04|    1|
|           -0.01|    1|
|             0.0|   11|
|            0.01|   12|
|            0.02|   17|
+----------------+-----+
only showing top 20 rows



##### D. Removing ads where Age of Car < 0

In [23]:
autos_DF =autos_DF.where('AgeofCar_inyears > 0')

##### E. Number of Ads left

In [24]:
display(Markdown("After Filtering, the dataset will be made of **%d rows**." % autos_DF.count()))

After Filtering, the dataset will be made of **244702 rows**.

## 8. Overview of  price vs count
##### A. Checking  the Cheapest, Expensive, Most sold, Least sold brands

In [25]:
brands = autos_DF.groupBy("brand")\
                     .agg(avg('Price').alias("AveragePrice"),\
                          min('Price').alias("MinPrice"),\
                          max('Price').alias("MaxPrice"),\
                         count('brand').alias('Count'))


brands = brands.withColumn('AveragePrice', sf.round('AveragePrice', 2))

print("Checking amount of distinct brands:")
autos_DF.select(countDistinct("brand").alias('brands')).show()

print("Checking the total number of brands sold by increasing order:")
brands.select('brand','Count').orderBy(asc('count')).show()

print("Checking the total number of brands sold by decreasing order:")
brands.select('brand','Count').orderBy(desc('count')).show()

print ("Brands ordered by Decreasing of Average Price:")
brands.orderBy(desc("AveragePrice")).select("brand","AveragePrice", "MinPrice", "MaxPrice").show()

print ("Brands ordered by Increasing of Minimum Price:")
brands.orderBy(asc("MinPrice")).select("brand","AveragePrice", "MinPrice", "MaxPrice").show()

print ("Brands ordered by Decreasing of Maximum Price:")
brands.orderBy(desc("MaxPrice")).select("brand","AveragePrice", "MinPrice", "MaxPrice").show()

CountofBrands = brands.select('brand','Count').orderBy(desc('count'))
BrandRatio = CountofBrands.withColumn('Ratio', (col("Count")/(239314))*100)
BrandRatio = BrandRatio.withColumn('Ratio', sf.round('Ratio',2))


print("Checking ratio of brands by decreasing order:")
BrandRatio.orderBy(desc("Ratio")).show()

print("Checking ratio of brands by increasing order:")
BrandRatio.orderBy(asc("Ratio")).show()



print ("Summary of the above:")
HighestAveragePrice    = brands.orderBy(desc("AveragePrice")).first()
LowestAveragePrice     = brands.orderBy(asc("AveragePrice")).first()
Mostsold               = brands.orderBy(desc('count')).first()
Leastsold              = brands.orderBy(asc('count')).first()
Mostsoldratio          = BrandRatio.orderBy(desc("Ratio")).first()
Leastsoldratio         = BrandRatio.orderBy(asc("Ratio")).first()


display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("Highest Average Price", "Lowest Average Price", "Most Sold Brand", "Least Sold Brand", \
       "%s (%s Euros)" % (HighestAveragePrice["brand"],HighestAveragePrice["AveragePrice"]), \
       "%s (%s Euros)" % (LowestAveragePrice["brand"],LowestAveragePrice["AveragePrice"]), \
       "%s (%s percent)" % (Mostsold["brand"], Mostsoldratio['Ratio'] ), \
       "%s (%s percent)" % (Leastsold["brand"], Leastsoldratio['Ratio']))))

       





Checking amount of distinct brands:
+------+
|brands|
+------+
|    40|
+------+

Checking the total number of brands sold by increasing order:
+----------+-----+
|     brand|Count|
+----------+-----+
|      lada|  136|
|   trabant|  164|
|     rover|  233|
|    lancia|  275|
|    daewoo|  279|
|      saab|  386|
|  daihatsu|  427|
|    jaguar|  475|
|    subaru|  484|
|      jeep|  602|
|land_rover|  626|
|     dacia|  720|
|  chrysler|  955|
| chevrolet| 1312|
|alfa_romeo| 1552|
|    suzuki| 1582|
|   porsche| 1715|
|     honda| 1797|
|mitsubishi| 1815|
|       kia| 1883|
+----------+-----+
only showing top 20 rows

Checking the total number of brands sold by decreasing order:
+--------------+-----+
|         brand|Count|
+--------------+-----+
|    volkswagen|50326|
|           bmw|29192|
| mercedes_benz|26173|
|          audi|23753|
|          opel|23167|
|          ford|15290|
|       renault|10116|
|       peugeot| 7323|
|          fiat| 5570|
|          seat| 4637|
|         sko


| Highest Average Price | Lowest Average Price | Most Sold Brand | Least Sold Brand |
|----|----|----|----|
| porsche (35776.57 Euros) | daewoo (1302.82 Euros) | volkswagen (21.03 percent) | lada (0.06 percent) |


##### B.  Filtering illogical high prices of cars

In [26]:
autos_DF = autos_DF.where("not (brand = 'nissan' and price = 99999)").where("not (brand = 'renault' and price = 93000)")\
.where("not (brand = 'smart' and price = 99999)")
autos_DF.orderBy(desc("Price")).limit(30).show(truncate=False, vertical=True)

-RECORD 0---------------------------------
 price              | 99999               
 vehicleType        | cabrio              
 gearbox            | automatik           
 kilometer          | 20000               
 fuelType           | benzin              
 brand              | porsche             
 notRepairedDamage  | nein                
 dateCreated        | 2016-03-10          
 lastSeen           | 2016-04-07 12:15:22 
 DateofRegistration | 2013-07-01          
 AgeofCar_inyears   | 2.69                
 Timeonline_indays  | 28                  
-RECORD 1---------------------------------
 price              | 99999               
 vehicleType        | coupe               
 gearbox            | manuell             
 kilometer          | 150000              
 fuelType           | benzin              
 brand              | porsche             
 notRepairedDamage  | ja                  
 dateCreated        | 2016-03-25          
 lastSeen           | 2016-04-06 14:44:52 
 DateofRegi

## 9. Overview of Timeonline vs Kilometers
#### A. Checking for Least and Most car online and distance covered

In [27]:
Timeonline_vs_Kilometer = autos_DF.groupBy('brand').agg(avg('Timeonline_indays').alias('AverageTimeonline'),\
                                                  avg('kilometer').alias('AverageKilometer'))


Timeonline_vs_Kilometer = Timeonline_vs_Kilometer.withColumn('AverageTimeonline', sf.round('AverageTimeonline', 2))
Timeonline_vs_Kilometer = Timeonline_vs_Kilometer.withColumn('AverageKilometer', sf.round('AverageKilometer', 2))

print("Checking average time of the brands kept online by decreasing order:")
Timeonline_vs_Kilometer.orderBy(desc("AverageTimeonline")).show()

print("Checking average time of a brand kept online by increasing order:")
Timeonline_vs_Kilometer.orderBy(asc("AverageTimeonline")).show()

print ("Summary of kilometer:")
autos_DF.select("kilometer").summary().show()

print ("Brands ordered by Decreasing of Average Distance covered:")
Timeonline_vs_Kilometer.orderBy(desc('AverageKilometer')).show()

print ("Brands ordered by Increasing of Average Distance covered:")
Timeonline_vs_Kilometer.orderBy(asc('AverageKilometer')).show()


LeastOnlineCar    = Timeonline_vs_Kilometer.orderBy(col("AverageTimeonline").asc()).first()
MostOnlineCar     = Timeonline_vs_Kilometer.orderBy(col("AverageTimeonline").desc()).first()
LeastCarDriven     = Timeonline_vs_Kilometer.orderBy(col("AverageKilometer").asc()).first()
MostCarDriven      = Timeonline_vs_Kilometer.orderBy(desc('AverageKilometer')).first()


print ("Summary of the above:")

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("Least Car Online", "Most Car Online", "Least Car Driven", "Most Car Driven", \
       "%s" % (LeastOnlineCar["brand"]), \
       "%s" % (MostOnlineCar["brand"]), \
       "%s" % (LeastCarDriven["brand"]), \
       "%s" % (MostCarDriven["brand"]))))



Checking average time of the brands kept online by decreasing order:
+--------------+-----------------+----------------+
|         brand|AverageTimeonline|AverageKilometer|
+--------------+-----------------+----------------+
|        jaguar|            11.32|       123178.95|
|        lancia|             11.2|       119963.64|
|sonstige_autos|            11.12|        93165.14|
|        subaru|            10.81|       122809.92|
|          mini|            10.58|        91992.64|
|          saab|            10.44|       141476.68|
|    land_rover|            10.38|       117867.41|
|    alfa_romeo|            10.32|       126459.41|
|         volvo|            10.08|        136870.0|
|       porsche|            10.04|       103373.18|
|          jeep|             9.86|       119368.77|
|         rover|              9.8|       133669.53|
|          audi|             9.79|        126615.8|
|     chevrolet|             9.73|        97995.43|
|         honda|             9.72|       122223


| Least Car Online | Most Car Online | Least Car Driven | Most Car Driven |
|----|----|----|----|
| daewoo | jaguar | trabant | saab |


## 10. Overview of Age vs Fuel Type
#### A. Checking for Newest and Oldest Car with the most used fuel types



In [28]:
AverageAgeofCar = autos_DF.groupBy('brand').agg(avg('AgeofCar_inyears').alias('AverageAgeofCar'), count('brand').alias("Count"))
                                                


AverageAgeofCar = AverageAgeofCar.withColumn('AverageAgeofCar', sf.round('AverageAgeofCar', 2))

print("Checking average age of the brands by decreasing order:")
AverageAgeofCar.select("brand", "AverageAgeofCar").orderBy(desc("AverageAgeofCar")).show()

print("Checking average age of the brands by increasing order:")
AverageAgeofCar.select("brand", "AverageAgeofCar").orderBy(asc("AverageAgeofCar")).show()

print("Most Common Fuel Types:")
autos_DF.groupBy('fuelType').agg(avg('Price').alias('Average Price'), count('fuelType').alias('Count'))\
                                                                                .orderBy(desc('Count')).show()


MostCommonFuel = autos_DF.groupBy('fuelType').agg(avg('Price').alias('Average Price'), count('fuelType').alias('Count'))\
                                                                                .orderBy(desc('Count'))


OldestCar                        = AverageAgeofCar.orderBy(desc("AverageAgeofCar")).first()
NewestCar                        = AverageAgeofCar.orderBy(asc("AverageAgeofCar")).first()
MostCommonFuelType               = MostCommonFuel.select('fuelType').first()
SecondMostCommonFuelType         = MostCommonFuel.select('fuelType').collect()[1]


print ("Summary of the above:")

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("Oldest Car", "Newest Car", "Most Common Fuel Type", "Second Most Common Fuel Type ", \
       "%s" % (OldestCar["brand"]), \
       "%s" % (NewestCar["brand"]), \
       "%s" % (MostCommonFuelType["fuelType"]), \
       "%s" % (SecondMostCommonFuelType["fuelType"]))))


Checking average age of the brands by decreasing order:
+--------------+---------------+
|         brand|AverageAgeofCar|
+--------------+---------------+
|       trabant|          32.12|
|sonstige_autos|          21.34|
|         rover|          16.33|
|          saab|          15.83|
|        lancia|           15.2|
|        jaguar|          14.72|
|          jeep|          14.26|
|         volvo|          13.95|
|      chrysler|          13.86|
|        subaru|           13.8|
|       porsche|          13.78|
|         honda|          13.54|
|        daewoo|          13.52|
| mercedes_benz|          13.33|
|    alfa_romeo|          13.25|
|    mitsubishi|          12.93|
|          opel|          12.82|
|      daihatsu|          12.57|
|    volkswagen|          12.39|
|          lada|          12.35|
+--------------+---------------+
only showing top 20 rows

Checking average age of the brands by increasing order:
+----------+---------------+
|     brand|AverageAgeofCar|
+----------+


| Oldest Car | Newest Car | Most Common Fuel Type | Second Most Common Fuel Type  |
|----|----|----|----|
| trabant | dacia | benzin | diesel |


#### B. Brands by Age

<div class = "alert alert-danger">
Nice that you have used pivot
</div>

In [29]:
AgeofCar = autos_DF.withColumn("AgeofCar",when((col("AgeofCar_inyears")>0) & (col("AgeofCar_inyears")<=5),"1.new")\
                               .when((col("AgeofCar_inyears")>5) & (col("AgeofCar_inyears")<=10),"2.medium")\
                               .when((col("AgeofCar_inyears")>10) & (col("AgeofCar_inyears")<=30),"3.old")\
                               .otherwise("4.classic"))

print("Top 20 brand with most new cars:")

AgeofCar.groupBy("brand").pivot("AgeofCar").count().orderBy(col("`1.new`").desc()).show()

print("Top 20 brand with most classic cars:")

AgeofCar.groupBy("brand").pivot("AgeofCar").count().orderBy(col("`4.classic`").desc()).show()

print("Ratio of Cars depending on Age:")
AgeofCar.groupBy("AgeofCar")\
                     .agg(count("AgeofCar").alias("TotalNumberofCars"), \
                          ((count("AgeofCar")/239314)*100).alias("Ratio"))\
                     .orderBy(desc("Ratio"))\
                     .select("AgeofCar","TotalNumberofCars",sf.round("Ratio",2).alias("Ratio")).show()

print("Ratio of Cars depending on Average Price:")
AgeofCar.groupBy("AgeofCar")\
                     .agg(count("AgeofCar").alias("TotalNumberofCars"), \
                           avg("price").alias("AveragePrice"))\
                     .orderBy(desc("AveragePrice"))\
                    .select("AgeofCar","TotalNumberofCars",sf.round("AveragePrice",2).alias("AveragePrice")).show()

Top 20 brand with most new cars:
+-------------+-----+--------+-----+---------+
|        brand|1.new|2.medium|3.old|4.classic|
+-------------+-----+--------+-----+---------+
|   volkswagen| 6638|   13336|29704|      648|
|         audi| 3805|    7595|12261|       92|
|          bmw| 2724|    8958|17327|      183|
|mercedes_benz| 2472|    6177|16767|      757|
|         opel| 1899|    5374|15639|      255|
|         ford| 1680|    4242| 9090|      278|
|        skoda| 1014|    1913| 1516|       11|
|         seat|  876|    1300| 2461|     null|
|      renault|  837|    2166| 7058|       54|
|      hyundai|  722|    1026|  973|     null|
|         fiat|  694|    1724| 2994|      158|
|         mini|  683|    1262|  896|       12|
|      peugeot|  569|    2212| 4523|       19|
|       nissan|  478|     858| 1924|       12|
|          kia|  469|     652|  762|     null|
|        smart|  466|    1089| 1743|     null|
|       toyota|  442|    1250| 1909|       27|
|      citroen|  363|    13

#### C. Fuel Type Average Price

In [30]:
autos_DF.groupBy('fuelType').agg(avg('Price').alias('AveragePrice'), count('fuelType').alias('Count')).orderBy(desc("AveragePrice")).show()


+--------+------------------+------+
|fuelType|      AveragePrice| Count|
+--------+------------------+------+
|  hybrid|13807.063829787234|   235|
| elektro|13044.260869565218|    69|
|  diesel| 9399.991486375862| 85510|
|  andere| 6303.242857142857|    70|
|  benzin| 5865.176060896357|154492|
|     cng|5175.7174392935985|   453|
|     lpg|  4900.20645994832|  3870|
+--------+------------------+------+



In [31]:
print("Checking types of brands that use hybrid as a fueltype:")

autos_DF.select('brand', 'Price','vehicleType', 'DateofRegistration').where(" fuelType == 'hybrid' ").orderBy(desc("Price")).show()

Checking types of brands that use hybrid as a fueltype:
+--------------+-----+-----------+------------------+
|         brand|Price|vehicleType|DateofRegistration|
+--------------+-----+-----------+------------------+
| mercedes_benz|62900|  limousine|        2014-11-01|
|         volvo|49700|      kombi|        2014-08-01|
|           bmw|48999|        suv|        2010-06-01|
|sonstige_autos|41900|  limousine|        2014-02-01|
|    mitsubishi|41800|  limousine|        2015-11-01|
|           bmw|38450|  limousine|        2012-01-01|
|          audi|37900|      kombi|        2015-01-01|
|           bmw|37750|        suv|        2010-03-01|
|           bmw|37750|  limousine|        2010-03-01|
|sonstige_autos|35250|        suv|        2011-03-01|
|sonstige_autos|33499|  limousine|        2008-02-01|
|    volkswagen|32500|  limousine|        2016-03-01|
|    volkswagen|32500|  limousine|        2015-07-01|
|sonstige_autos|32000|  limousine|        2014-02-01|
|          audi|31250|  li

#### D. Count of different types of gearbox

In [32]:
print("Checking different types of gearbox:")
autos_DF.groupBy('gearbox').count().show()

Checking different types of gearbox:
+---------+------+
|  gearbox| count|
+---------+------+
|automatik| 59919|
|  manuell|184780|
+---------+------+

